In [1]:
# imports
import os
import certifi
import arcpy

In [ ]:
# set default gdal and certifi envs... improve?
os.environ['GDAL_DATA']  = r'C:\Program Files\ArcGIS\Pro\Resources\pedata\gdaldata'
os.environ.setdefault("CURL_CA_BUNDLE", certifi.where())

In [2]:
arcpy.GetInstallInfo()

{'LicenseLevel': 'Advanced', 'InstallDir': 'c:\\program files\\arcgis\\pro\\', 'Installer': '262272G', 'ProductName': 'ArcGISPro', 'Version': '2.8', 'SourceDir': 'C:\\Users\\262272G\\Documents\\ArcGIS Pro 2.8\\ArcGISPro\\', 'InstallType': 'N/A', 'BuildNumber': '29751', 'InstallDate': '26/07/2021', 'InstallTime': '21:40:52', 'SPNumber': 'N/A', 'SPBuild': 'N/A'}

In [8]:
os.path.join(install_dir, 'Resources\pedata\gdaldata')

'c:\\program files\\arcgis\\pro\\Resources\\pedata\\gdaldata'